In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def basis_function(i, k, t, knot_vector):
    """ Рекурсивно вычисляет значения i-й B-сплайн базисной функции k-го порядка в точке t """
    if k == 1:
        return 1.0 if (knot_vector[i] <= t < knot_vector[i + 1]) or \
                       (t == knot_vector[-1] and knot_vector[i] <= t) else 0.0
    else:
        term1 = 0.0
        if knot_vector[i + k - 1] != knot_vector[i]:
            term1 = (t - knot_vector[i]) / (knot_vector[i + k - 1] - knot_vector[i]) * \
                    basis_function(i, k - 1, t, knot_vector)

        term2 = 0.0
        if knot_vector[i + k] != knot_vector[i + 1]:
            term2 = (knot_vector[i + k] - t) / (knot_vector[i + k] - knot_vector[i + 1]) * \
                    basis_function(i + 1, k - 1, t, knot_vector)

        return term1 + term2

def nurbs_curve(control_points, weights, knot_vector, k, num_points=100):
    """ Вычисляет NURB-кривую по контрольным точкам, весам, узловому вектору и степени """
    n = len(control_points) - 1
    curve = np.zeros((num_points, 2))
    t_values = np.linspace(knot_vector[k - 1], knot_vector[n + 1], num_points)

    for i in range(num_points):
        t = t_values[i]
        numerator = np.zeros(2)
        denominator = 0.0

        for j in range(n + 1):
            N_j_k_t = basis_function(j, k, t, knot_vector)
            numerator += N_j_k_t * weights[j] * np.array(control_points[j])
            denominator += N_j_k_t * weights[j]

        curve[i] = numerator / denominator if denominator != 0 else np.zeros(2)

    return curve

# Начальные контрольные точки и веса
control_points = np.array([[0, 0], [1, 2], [2, -1], [3, 3], [4, 0], [5, 2], [6, 1]])
weights = np.array([1, 2, 1, 2, 1, 2, 1])

# Узловой вектор
k = 3
knot_vector = np.array([0, 0, 0, 1, 2, 3, 3, 4, 4, 4])

# Создание виджетов для контрольных точек и весов
points_widgets = [[widgets.FloatSlider(min=-5, max=5, value=point[dim], description=f'P{i}{dim}')
                   for dim in range(2)] for i, point in enumerate(control_points)]
weights_widgets = [widgets.FloatSlider(min=0.1, max=5, value=w, description=f'W{i}') 
                   for i, w in enumerate(weights)]

# Функция для отображения графика
def display_curve(**kwargs):
    # Извлечение контрольных точек и весов из kwargs
    num_points = len(control_points)
    new_control_points = np.array([kwargs[f'point_{i}_{dim}'] for i in range(num_points) for dim in range(2)]).reshape(num_points, 2)
    new_weights = np.array([kwargs[f'weight_{i}'] for i in range(num_points)])

    # Расчет и отображение NURB-кривой
    nurbs = nurbs_curve(new_control_points, new_weights, knot_vector, k)
    plt.plot(nurbs[:, 0], nurbs[:, 1], label="NURB-кривая", color='blue')
    plt.plot(new_control_points[:, 0], new_control_points[:, 1], 'ro--', label="Контрольные точки")
    plt.legend()
    plt.grid(True)
    plt.show()


# Создание layout для группировки виджетов
points_flat = [item for sublist in points_widgets for item in sublist]
widget_dict = {f'point_{i}_{dim}': widget for i, sublist in enumerate(points_widgets) 
               for dim, widget in enumerate(sublist)}
widget_dict.update({f'weight_{i}': weights_widgets[i] for i in range(len(weights_widgets))})

# Создание интерактивного вывода
interactive_plot = widgets.interactive_output(display_curve, widget_dict)


# Отображение виджетов и интерактивного вывода
points_box = widgets.VBox([widgets.HBox(sublist) for sublist in points_widgets])
weights_box = widgets.VBox(weights_widgets)
controls_box = widgets.HBox([points_box, weights_box])
display(controls_box, interactive_plot)


Output()